# <img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=400 align="left"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=400 align="left"/><br><br><br><br><br>



# Prepare ALOS2 data and generate L2 NISAR-like InSAR data products

(*This workflow was adapted from Alex C.'s GCOV workflow, for InSAR products. For questions, reach out to Catalina T.*)

In preparation for NISAR, this notebook **converts ALOS-2 SLC data from ASF to NISAR RSLC and generates NISAR L1 and L2 sample data products using ISCE3 InSAR (e.g. RUNW, GUNW)**:
1. Downloads desired ALOS-2 pair from ASF (note these data are restricted under NASA-JAXA agreement)
2. Converts ALOS2 SLC to NISAR RSLC using `alos2_to_nisar_l1.py`
3. Processes a DEM for the study area using the `stage_dem.py` from isce3
4. Processes NISAR RSLC to GUNW using `insar.py` from isce3 and a `runconfig_insar.yaml` ← change parameters for custom products
- `runconfig_insar_VB_template.yaml` ← standard unwrapping 80m
- `runconfig_insar_20m_unw.yaml` ← unwrap at 20m

Notes:
- This workflow example is designed to process a single pair at a time. Granule ID should be known ahead of time and used for folder structure. e.g. pair ALOS2456335530-221104_ALOS2458405530-221118
- For a PCM job, workflow may need to be adjusted to optimize pairing up and processing.
- Current workflow runs on `/scratch` but if it's possible to stage all files on `s3` and link from there, that might be better since `s3` is cheaper and backed up compared to `/scratch`.

In [1]:
import os
import sys
from pathlib import Path
import subprocess
import fnmatch
import zipfile
import h5py
from string import Template

#### Choose your place to save your outputs. 

`/scratch` can be used for short-term storage, but the cost is much higher and the space is not backed up. Files for long-term storage should be moved to an S3 bucket.

In [2]:
scratch_folder = Path('/scratch/taglialatela_cryo_test/sample_products/ALOS2/')   ##change to your folder

#### Choose your AOI

New directories will be made to store files for this AOI

In [3]:
aoi = 'antarctica'

aoi_dir = scratch_folder/aoi
PAIR_dir = aoi_dir / 'ALOS2456335530-221104-ALOS2458405530-221118'   #manual update based on granuleIDs of pair to process
ALOS2_dir1 = aoi_dir / PAIR_dir / 'ALOS2_ref'
ALOS2_dir2 = aoi_dir / PAIR_dir / 'ALOS2_sec'
DEM_dir = aoi_dir / PAIR_dir / 'DEM'
INSAR_dir = aoi_dir / PAIR_dir / 'INSAR'
unwrap_80m_dir = aoi_dir / PAIR_dir / INSAR_dir / 'unwrap_80m'
output_insar_dir = aoi_dir / PAIR_dir / INSAR_dir / unwrap_80m_dir / 'output_insar'
scratch_insar_dir = aoi_dir / PAIR_dir / INSAR_dir / unwrap_80m_dir / 'scratch_insar'
RSLC_dir1 = aoi_dir /  PAIR_dir / 'RSLC_ref'
RSLC_dir2 = aoi_dir /  PAIR_dir / 'RSLC_sec'

Path(aoi_dir).mkdir(parents=True, exist_ok=True)
Path(PAIR_dir).mkdir(parents=True, exist_ok=True)
Path(ALOS2_dir1).mkdir(parents=True, exist_ok=True)
Path(ALOS2_dir2).mkdir(parents=True, exist_ok=True)
Path(DEM_dir).mkdir(parents=True, exist_ok=True)
Path(INSAR_dir).mkdir(parents=True, exist_ok=True)
Path(unwrap_80m_dir).mkdir(parents=True, exist_ok=True)
Path(output_insar_dir).mkdir(parents=True, exist_ok=True)
Path(scratch_insar_dir).mkdir(parents=True, exist_ok=True)
Path(RSLC_dir1).mkdir(parents=True, exist_ok=True)
Path(RSLC_dir2).mkdir(parents=True, exist_ok=True)

_**Skip the download steps if already downloaded** these files previsouly. e.g. provide s3 link to ALOS2 data in NISAR ODS instead._

#### Copy the ALOS2 URLs into a list

Workflow build to run a single pair (2 frames) at a time. Here we define the 2 frames for the pair we want to process, by providing the ASF link for download. Corresponding links are in the ASF-ALOS2 spreadsheet. 

In [4]:
ALOS_SLC_ref = 'https://cumulus.asf.alaska.edu/L1.1/A4/0000487643_001001_ALOS2456335530-221104.zip'
ALOS_SLC_sec = 'https://cumulus.asf.alaska.edu/L1.1/A4/0000489662_001001_ALOS2458405530-221118.zip'

In [5]:
print(ALOS_SLC_ref)
print(ALOS_SLC_sec)

https://cumulus.asf.alaska.edu/L1.1/A4/0000487643_001001_ALOS2456335530-221104.zip
https://cumulus.asf.alaska.edu/L1.1/A4/0000489662_001001_ALOS2458405530-221118.zip


#### Download the ALOS2 files using wget

Download 1 pair at a time. 

This seems to take some time. Single zip around 6 GB

In [6]:
filename1 = ALOS2_dir1 / ALOS_SLC_ref.split('/')[-1]
if os.path.isfile(filename1)==False:
    command = f"wget -P {ALOS2_dir1} -q {ALOS_SLC_ref}"
    output = subprocess.check_output(command, shell=True)    

In [7]:
filename2 = ALOS2_dir2 / ALOS_SLC_sec.split('/')[-1]
if os.path.isfile(filename2)==False:
    command = f"wget -P {ALOS2_dir2} -q {ALOS_SLC_sec}"
    output = subprocess.check_output(command, shell=True)

#### Unzip the ALOS2 files

Note: `alos2_to_nisar_l1.py` assumes there is only one directory or file it is pointing to. The zip file should be unzipped individually and left in their own directory.

If you don't want to keep the zip files, you should uncomment the `os.remove` line

##### (a) Unzip the *reference* ALOS2 data in its own folder:

In [9]:
zipfiles = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir1)
                for f in fnmatch.filter(files,'*.zip')]
for file in zipfiles:
    print(file)
    if os.path.isdir(ALOS2_dir1 / file[:-4]):
        ##check full unzipping
        if sum(os.path.getsize(ALOS2_dir1 / file[:-4] / f) for f in os.listdir(ALOS2_dir1 / file[:-4]) if os.path.isfile(ALOS2_dir1 / file[:-4] /f)) < 6509714637:
            with zipfile.ZipFile(file, 'r') as zip_ref:
                zip_ref.extractall(ALOS2_dir1/file.split('/')[-1][:-4])
            os.remove(file)
    else:
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(ALOS2_dir1/file.split('/')[-1][:-4])
            os.remove(file)

/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/ALOS2_ref/0000487643_001001_ALOS2456335530-221104.zip


##### (b) Unzip the *secondary* ALOS2 data in its own folder:

In [10]:
zipfiles = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir2)
                for f in fnmatch.filter(files,'*.zip')]
for file in zipfiles:
    print(file)
    # if os.path.isdir(ALOS2_dir2 / file[:-4]):
        # ##check full unzipping
        # if sum(os.path.getsize(ALOS2_dir2 / file[:-4] / f) for f in os.listdir(ALOS2_dir2 / file[:-4]) if os.path.isfile(ALOS2_dir1 / file[:-4] /f)) < 6509714637:
        #     with zipfile.ZipFile(file, 'r') as zip_ref:
        #         zip_ref.extractall(ALOS2_dir2/file.split('/')[-1][:-4])
        #     # os.remove(file)
    # else:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(ALOS2_dir2/file.split('/')[-1][:-4])
        os.remove(file)

/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/ALOS2_sec/0000489662_001001_ALOS2458405530-221118.zip


##### (c) Get a list of folders for each ALOS2 file

_If previously downloaded ALOS2 data and stored in NISAR ODS (and you skipped the steps above), this is where you can provide the path to the s3 bucket (untested) for ALOS2 unzipped data._

In [6]:
ALOS2folder1 = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir1)
                for f in fnmatch.filter(dirnames,'*')]      # all directories
                # for f in fnmatch.filter(dirnames,'*ALOS2*')]   # ALOS2 data directories only
ALOS2folder2 = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir2)
                for f in fnmatch.filter(dirnames,'*')]         # all directories
print(ALOS2folder1)
print(ALOS2folder2)

['/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/ALOS2_ref/0000487643_001001_ALOS2456335530-221104']
['/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/ALOS2_sec/0000489662_001001_ALOS2458405530-221118']


### Loop through each ALOS2 file and:
##### 1. **Convert ALOS2 SLC to NISAR RSLC**

For ALOS-2 data we can only start from SLC (L1.1) and repackage them in a NISAR RSLC HDF5 format. This is the only SLC format recognized by the NISAR workflow in ISCE3. The conversion script is available here https://github.com/isce-framework/isce3/blob/develop/share/nisar/examples/alos2_to_nisar_l1.py

##### 2. **Get a DEM for the study area** using the `stage_dem.py` from isce3
##### 3. **Process NISAR RSLC to GUNW** using `insar.py` from isce3 and a `runconfig_insar.yaml` (<-- To create custom L2 products or troubleshoot, edit paramters in this file.)

--------------

**(1a) Process Ref frame first:**

In [7]:
# Convert ALOS2 to NISAR-like format:
ALOS2_id1 = ALOS2folder1[0].split('/')[-1]
print('')
print(ALOS2_id1)  

# Comment the next section out if you've already converted, and just need to define paramters for later steps in workflow
# command = f"conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder} -o {RSLC_dir/ALOS2_id}.h5"
command = f"conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder1[0]} -o {RSLC_dir1/ALOS2_id1}.h5"
print('')
print(command)    
if os.path.isfile(f"{RSLC_dir1/ALOS2_id1}.h5")==False or os.path.getsize(f"{RSLC_dir1/ALOS2_id1}.h5")<6000000000:
    try: os.remove(f"{RSLC_dir1/ALOS2_id1}.h5")
    except:''
    output = subprocess.check_output(command, shell=True)


0000487643_001001_ALOS2456335530-221104

conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i /scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/ALOS2_ref/0000487643_001001_ALOS2456335530-221104 -o /scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/RSLC_ref/0000487643_001001_ALOS2456335530-221104.h5


overwriting variable {'LD_LIBRARY_PATH'}



**(1b) Process secondary frame next** (no DEM step here since we only need one, and we used the ref data to make it):

In [9]:
ALOS2_id2 = ALOS2folder2[0].split('/')[-1]
print('')
print(ALOS2_id2)  
    
# command = f"conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder} -o {RSLC_dir/ALOS2_id}.h5"
command = f"conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder2[0]} -o {RSLC_dir2/ALOS2_id2}.h5"
print('')
print(command)    
if os.path.isfile(f"{RSLC_dir2/ALOS2_id2}.h5")==False or os.path.getsize(f"{RSLC_dir2/ALOS2_id2}.h5")<6000000000:
    try: os.remove(f"{RSLC_dir2/ALOS2_id2}.h5")
    except:''
    output = subprocess.check_output(command, shell=True)


0000489662_001001_ALOS2458405530-221118

conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i /scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/ALOS2_sec/0000489662_001001_ALOS2458405530-221118 -o /scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/RSLC_sec/0000489662_001001_ALOS2458405530-221118.h5


overwriting variable {'LD_LIBRARY_PATH'}



**(2) Get a DEM for the study area** using the `stage_dem.py` from isce3 and the reference RSLC we made above. 

In [8]:
## Get NISAR DEM
command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p {RSLC_dir1/ALOS2_id1}.h5 -o {DEM_dir/ALOS2_id1}.vrt"
print('')
print(command)    

if os.path.isfile(f"{DEM_dir/ALOS2_id1}.vrt")==False:
    output = subprocess.check_output(command, shell=True)


conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/RSLC_ref/0000487643_001001_ALOS2456335530-221104.h5 -o /scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/DEM/0000487643_001001_ALOS2456335530-221104.vrt


### Generate NISAR-like INSAR products**

Set up the runconfig with right paths for pair to be processed.

In [10]:
print(RSLC_dir1)
print(ALOS2_id1)
print(DEM_dir)
print(PAIR_dir)
print(output_insar_dir)

/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/RSLC_ref
0000487643_001001_ALOS2456335530-221104
/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/DEM
/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118
/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/INSAR/unwrap_80m/output_insar


Provide path to the runconfig used by isce3 code

In [11]:
filein = open('/home/jovyan/cryo_sample_products/runconfig_insar_VB_template.yaml')
template = Template(filein.read())
replacements = {'inputrefrslc':  f"{RSLC_dir1/ALOS2_id1}.h5",
                'inputsecrslc':  f"{RSLC_dir2/ALOS2_id2}.h5",
                'demfile': f"{DEM_dir/ALOS2_id1}.vrt",
                'prodpath': f"{output_insar_dir}",
                'scratchpath': f"{scratch_insar_dir}",
                'sasoutpath':  f"{output_insar_dir/ALOS2_id1}_product.h5",
                'logpath': f"{output_insar_dir}/insar.log"
                }
makeoutput = template.substitute(replacements)
file = open('%s/%s.yaml' %(output_insar_dir,ALOS2_id1),'w')
file.write(makeoutput)
file.close()
filein.close()

##### NISAR RSLC --> NISAR L1 and L2 insar products

In [12]:
command = f"conda run -n isce3_src python /home/jovyan/isce3/python/packages/nisar/workflows/insar.py {output_insar_dir/ALOS2_id1}.yaml"
print('')
print(command)
# if os.path.isfile(f"{output_insar_dir/ALOS2_id1}_product.h5")==False:
if os.path.isfile(f"{output_insar_dir}_product.h5")==False:  #this might be right syntax to get e.g. ROFF_product.h5 which might be standard naming
    output = subprocess.check_output(command, shell=True)


conda run -n isce3_src python /home/jovyan/isce3/python/packages/nisar/workflows/insar.py /scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/INSAR/unwrap_80m/output_insar/0000487643_001001_ALOS2456335530-221104.yaml


overwriting variable {'LD_LIBRARY_PATH'}
/home/jovyan/isce3/build/packages/nisar/workflows/offsets_product.py:86: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'a', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/rubbersheet.py:73: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'r+', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/unwrap.py:65: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, "a", libver="latest", swmr=True) as dst_h5,\
/home/jovyan/isce3/build/packages/nisar/workflows/ionosphere.py:1055: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True

See final list of NISAR data files

In [13]:
NISAR_GUNWs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(output_insar_dir)
                for f in fnmatch.filter(files,'*.h5')]
NISAR_GUNWs

['/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/INSAR/unwrap_80m/output_insar/RUNW_0000487643_001001_ALOS2456335530-221104_product.h5',
 '/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/INSAR/unwrap_80m/output_insar/RIFG_0000487643_001001_ALOS2456335530-221104_product.h5',
 '/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/INSAR/unwrap_80m/output_insar/GOFF_0000487643_001001_ALOS2456335530-221104_product.h5',
 '/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/INSAR/unwrap_80m/output_insar/ROFF_0000487643_001001_ALOS2456335530-221104_product.h5',
 '/scratch/taglialatela_cryo_test/sample_products/ALOS2/antarctica/ALOS2456335530-221104-ALOS2458405530-221118/INSAR/unwrap_80m/output_insar/GUNW_0000487643_001001_ALOS2456335530-221104_product.h5']